In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect('create.db')
print("Opened database seccessfully")

cur = conn.cursor()

cur.execute('''
    CREATE TABLE COMPANY(
        ID INT PRIMARY KEY NOT NULL,
        NAME TEXT NOT NULL,
        AGE INT NOT NULL,
        ADDRESS CHAR(50),
        SALARY REAL);
    ''')
print("Table created successfully")

Opened database seccessfully
Table created successfully


In [3]:
cur.execute("INSERT INTO COMPANY (ID,NAME,AGE,ADDRESS,SALARY) VALUES (1, 'Paul', 32, 'California', 20000.00)");
cur.lastrowid, conn.total_changes

(1, 1)

In [5]:
conn.commit()

어느 정도 transaction으로 묶어 놓고 다 끝날 때까지 기다린 다음 commit 해야 일관성 문제 해결.  
문제 생기면 롤백

In [6]:
cur.execute("INSERT INTO COMPANY (ID,NAME,AGE,ADDRESS,SALARY) VALUES (:id, :name, :age, :address, :salary)",
            {'id':2, 'name':'Allen', 'age':25, 'address':'Texas', 'salary':15000.00});
data = [(3,'Teddy',23,'Norway',20000.00),
       (4, 'Mark',25,'Rich-Mond',65000.00)]
cur.executemany("INSERT INTO COMPANY (ID, NAME, AGE, ADDRESS, SALARY) VALUES (?,?,?,?,?)",data);

In [7]:
cur.lastrowid, conn.total_changes

(2, 4)

In [8]:
conn.commit()

In [20]:
cur.executescript("""
    INSERT INTO COMPANY (ID, NAME, AGE, ADDRESS, SALARY)
    VALUES (5, 'Mark1', 25, 'Rich-Mond', 65000.00);
    
    INSERT INTO COMPANY (ID, NAME, AGE, ADDRESS, SALARY)
    VALUES (6, 'Mark2', 25, 'Rich-Mond', 65000.00);
    """);

IntegrityError: UNIQUE constraint failed: COMPANY.ID

알아서 커밋

In [11]:
cur.lastrowid, conn.total_changes

(2, 6)

In [16]:
cur.execute('select * from company')
cur.fetchall()

[(1, 'Paul', 32, 'California', 20000.0),
 (2, 'Allen', 25, 'Texas', 15000.0),
 (3, 'Teddy', 23, 'Norway', 20000.0),
 (4, 'Mark', 25, 'Rich-Mond', 65000.0),
 (5, 'Mark1', 25, 'Rich-Mond', 65000.0),
 (6, 'Mark2', 25, 'Rich-Mond', 65000.0)]

In [17]:
cur.execute('select * from company')
for row in cur:
    print(row)

(1, 'Paul', 32, 'California', 20000.0)
(2, 'Allen', 25, 'Texas', 15000.0)
(3, 'Teddy', 23, 'Norway', 20000.0)
(4, 'Mark', 25, 'Rich-Mond', 65000.0)
(5, 'Mark1', 25, 'Rich-Mond', 65000.0)
(6, 'Mark2', 25, 'Rich-Mond', 65000.0)


In [18]:
type(row)

tuple

In [19]:
cid = 1

cur.execute("UPDATE COMPANY set SALARY = 25000.00 where ID = :id", {'id':cid})

In [21]:
conn.commit()

In [22]:
print("Total number of rows updated : ", conn.total_changes)

Total number of rows updated :  7


In [24]:
cur.execute("SELECT id, name, address, salary from COMPANY")
for row in cur:
    print(row)

(1, 'Paul', 'California', 25000.0)
(2, 'Allen', 'Texas', 15000.0)
(3, 'Teddy', 'Norway', 20000.0)
(4, 'Mark', 'Rich-Mond', 65000.0)
(5, 'Mark1', 'Rich-Mond', 65000.0)
(6, 'Mark2', 'Rich-Mond', 65000.0)


In [25]:
cur.execute("DELETE from COMPANY where ID = 2;")

print("Total number of rows deleted : ", conn.total_changes)

Total number of rows deleted :  8


ORM으로 넘어가면 자동으로 커밋시켜줌

In [26]:
cur.execute("SELECT id, name, address, salary from COMPANY")
for row in cur:
    print(row)

(1, 'Paul', 'California', 25000.0)
(3, 'Teddy', 'Norway', 20000.0)
(4, 'Mark', 'Rich-Mond', 65000.0)
(5, 'Mark1', 'Rich-Mond', 65000.0)
(6, 'Mark2', 'Rich-Mond', 65000.0)


In [27]:
conn.commit()

하나의 db에 connection을 여러개 열면 lock걸림 -> try except finally 구문으로 error handling

with conn: 으로 하면 자동적으로 conn.close() 된다

dump되면 textfile이 하나 생기는데 그 텍스트파일 안에는 sql 문

In [28]:
cur.executescript('''
    CREATE TABLE ARTIST(
        id INTEGER NOT NULL PRIMARY kEY AUTOINCREMENT UNIQUE,
        name TEXT NOT NULL);
    create table GENRE(
        id INTEGER NOT NULL PRIMARY kEY AUTOINCREMENT UNIQUE,
        name TEXT NOT NULL);
    create table ALBUM(
        id INTEGER NOT NULL PRIMARY kEY AUTOINCREMENT UNIQUE,
        title TEXT NOT NULL,
        artist_id INTEGER NOT NULL);
    ''')

In [30]:
cur.executescript('''
    CREATE TABLE Track(
        id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
        title TEXT,
        length INTEGER,
        rating INTEGER,
        count INTEGER,
        album_id INTEGER,
        genre_id INTEGER);
    insert into ARTIST (name) values ('Led Zepplin');
    insert into ARTIST (name) values ('AC/DC');
    insert into GENRE (name) values ('Rock');
    insert into GENRE (name) values ('Metal');
    ''')

In [34]:
cur.execute("""
    INSERT into album (title, artist_id)
    VALUES(:album,
    (SELECT id
    FROM artist 
    WHERE name=:artist))
    """, {"album": "thisalbum", "artist":'Led Zepplin'})

In [35]:
conn.commit()

In [36]:
cur.executescript(
"""
insert into ALBUM (title, artist_id) values ('Who Made Who', 2);
insert into ALBUM (title, artist_id) values ('IV', 1);
insert into track (title, rating, length, count, album_id, genre_id) values ('Black Dog', 5, 297, 0, 2, 1);
insert into track (title, rating, length, count, album_id, genre_id) values ('Stairway', 5, 482, 0, 2, 1);
insert into track (title, rating, length, count, album_id, genre_id) values ('About to Rock', 5, 313, 0, 1, 2);
insert into track (title, rating, length, count, album_id, genre_id) values ('Who Made Who', 5, 207, 0, 1, 2);
""")

In [38]:
cur.execute("SELECT * from album")
for row in cur:
    print(row)

(1, 'thisalbum', 1)
(2, 'Who Made Who', 2)
(3, 'IV', 1)


In [39]:
cur = conn.cursor()

In [40]:
cur.execute("SELECT * from album")
for row in cur:
    print(row)

(1, 'thisalbum', 1)
(2, 'Who Made Who', 2)
(3, 'IV', 1)


In [41]:
cur.execute('drop table album')

In [42]:
cur.execute("""
        create table ALBUM(
        id INTEGER NOT NULL PRIMARY kEY AUTOINCREMENT UNIQUE,
        title TEXT NOT NULL,
        artist_id INTEGER NOT NULL);
        """)

In [44]:
cur.executescript("""
insert into ALBUM (title, artist_id) values ('Who Made Who', 2);
insert into ALBUM (title, artist_id) values ('IV', 1);""")

In [45]:
cur.execute("SELECT * from album")
for row in cur:
    print(row)

(1, 'Who Made Who', 2)
(2, 'IV', 1)


In [46]:
cur.execute(
"""
select album.title, artist.name
from album
join artist on album.artist_id = artist.id;
""")
for row in cur:
    print(row)

('Who Made Who', 'AC/DC')
('IV', 'Led Zepplin')


In [49]:
cur.execute(
"""
select track.title, genre.name
from track
join genre on track.genre_id = genre.id;
""")
for row in cur:
    print(row)

('Black Dog', 'Rock')
('Stairway', 'Rock')
('About to Rock', 'Metal')
('Who Made Who', 'Metal')


In [50]:
cur.execute(
"""
select album.title, track.title
from track
join album
on track.album_id = album.id;
""")
for row in cur:
    print(row)

('IV', 'Black Dog')
('IV', 'Stairway')
('Who Made Who', 'About to Rock')
('Who Made Who', 'Who Made Who')


In [51]:
cur.execute(
"""
select *
from track
join album
on track.album_id = album.id;
""")
for row in cur:
    print(row)

(1, 'Black Dog', 297, 5, 0, 2, 1, 2, 'IV', 1)
(2, 'Stairway', 482, 5, 0, 2, 1, 2, 'IV', 1)
(3, 'About to Rock', 313, 5, 0, 1, 2, 1, 'Who Made Who', 2)
(4, 'Who Made Who', 207, 5, 0, 1, 2, 1, 'Who Made Who', 2)


In [53]:
cur.execute(
"""
select track.title, artist.name, album.title, genre.name
from track
join artist join album join genre
on track.album_id = album.id
    and track.genre_id = genre.id
    and album.artist_id = artist.id;
""")
for row in cur:
    print(row)

('Black Dog', 'Led Zepplin', 'IV', 'Rock')
('Stairway', 'Led Zepplin', 'IV', 'Rock')
('About to Rock', 'AC/DC', 'Who Made Who', 'Metal')
('Who Made Who', 'AC/DC', 'Who Made Who', 'Metal')
